In [27]:
import sympy as sp

In [28]:
om, nu, E_0, M, c_ref, D_0, R_g, T, R_0, eta_E, chi_max = sp.symbols("Omega, nu, E_0, M, c_ref, D_0, R_g, T, R_0, eta_E, chi_max")
rt, tt = sp.symbols("x, ttilde") # rt is non dimensional radius rtilde

ut = sp.Function('u')(rt, tt) # ut is non-dimensional but i have not taken variable name as utilde so that I can easily make it compatible with COMSOL variable name
ct = sp.Function('c')(rt, tt) # ct is non-dimensional as well, 

epsilon_rr = sp.diff(ut, rt, 1)
epsilon_tt = ut/rt 
epsilon_zz = 0

sigmat_rr = sp.Function("sigmatilde_rr")(rt, tt)
sigmat_tt = sp.Function("sigmatilde_theta_theta")(rt, tt)
sigmat_zz = sp.Function("sigmatilde_zz")(rt, tt)



In [29]:
# equilibrium equations in non dimensional form
expr1 = epsilon_rr - ((sigmat_rr - nu*(sigmat_tt+sigmat_zz))/(1+eta_E*chi_max*ct)+ ct/3)*om*c_ref
expr2 = epsilon_tt - ((sigmat_tt - nu*(sigmat_rr+sigmat_zz))/(1+eta_E*chi_max*ct)+ ct/3)*om*c_ref
expr3 = epsilon_zz - ((sigmat_zz - nu*(sigmat_tt+sigmat_rr))/(1+eta_E*chi_max*ct)+ ct/3)*om*c_ref

eq1 = sp.Eq(expr1, 0)
eq2 = sp.Eq(expr2, 0)
eq3 = sp.Eq(expr3, 0)

eq1

Eq(-Omega*c_ref*((-nu*(sigmatilde_theta_theta(x, ttilde) + sigmatilde_zz(x, ttilde)) + sigmatilde_rr(x, ttilde))/(chi_max*eta_E*c(x, ttilde) + 1) + c(x, ttilde)/3) + Derivative(u(x, ttilde), x), 0)

In [30]:
sigma = sp.linsolve([eq1, eq2, eq3], sigmat_rr, sigmat_tt, sigmat_zz)
sp.simplify(sigma)
sigma = sp.factor(sigma)
sigma = sigma.args[0]

In [32]:
sigmat_rr = sp.factor(sp.simplify(sigma[0]))
sigmat_tt = sp.factor(sp.simplify(sigma[1]))
sigmat_zz = sp.factor(sp.simplify(sigma[2]))
sp.factor(sigmat_rr)

(chi_max*eta_E*c(x, ttilde) + 1)*(Omega*c_ref*nu*x*c(x, ttilde) + Omega*c_ref*x*c(x, ttilde) + 3*nu*x*Derivative(u(x, ttilde), x) - 3*nu*u(x, ttilde) - 3*x*Derivative(u(x, ttilde), x))/(3*Omega*c_ref*x*(nu + 1)*(2*nu - 1))

In [33]:
sigmat_hyd = (sigmat_rr + sigmat_tt + sigmat_zz)/3
sp.factor(sp.simplify(sigmat_hyd))

-(chi_max*eta_E*c(x, ttilde) + 1)*(-Omega*c_ref*x*c(x, ttilde) + x*Derivative(u(x, ttilde), x) + u(x, ttilde))/(3*Omega*c_ref*x*(2*nu - 1))

# First PDE

In [34]:
pde1_lhs = sp.Eq(sp.diff(sigmat_rr, rt, 1) + (sigmat_rr - sigmat_tt)/rt, 0).lhs
sp.factor(sp.simplify(sp.factor(pde1_lhs)))

(2*Omega*c_ref*chi_max*eta_E*nu*x**2*c(x, ttilde)*Derivative(c(x, ttilde), x) + 2*Omega*c_ref*chi_max*eta_E*x**2*c(x, ttilde)*Derivative(c(x, ttilde), x) + Omega*c_ref*nu*x**2*Derivative(c(x, ttilde), x) + Omega*c_ref*x**2*Derivative(c(x, ttilde), x) + 3*chi_max*eta_E*nu*x**2*c(x, ttilde)*Derivative(u(x, ttilde), (x, 2)) + 3*chi_max*eta_E*nu*x**2*Derivative(c(x, ttilde), x)*Derivative(u(x, ttilde), x) + 3*chi_max*eta_E*nu*x*c(x, ttilde)*Derivative(u(x, ttilde), x) - 3*chi_max*eta_E*nu*x*u(x, ttilde)*Derivative(c(x, ttilde), x) - 3*chi_max*eta_E*nu*c(x, ttilde)*u(x, ttilde) - 3*chi_max*eta_E*x**2*c(x, ttilde)*Derivative(u(x, ttilde), (x, 2)) - 3*chi_max*eta_E*x**2*Derivative(c(x, ttilde), x)*Derivative(u(x, ttilde), x) - 3*chi_max*eta_E*x*c(x, ttilde)*Derivative(u(x, ttilde), x) + 3*chi_max*eta_E*c(x, ttilde)*u(x, ttilde) + 3*nu*x**2*Derivative(u(x, ttilde), (x, 2)) + 3*nu*x*Derivative(u(x, ttilde), x) - 3*nu*u(x, ttilde) - 3*x**2*Derivative(u(x, ttilde), (x, 2)) - 3*x*Derivative(u(x, t

In [35]:
# I have taken gamma1 to be sigmat_rr so that I can easily impose boudary condition at rt=1 on ut
Gamma1 = sigmat_rr
f1 = sp.diff(Gamma1, rt, 1) - pde1_lhs

In [36]:
Gamma1 = sp.factor(sp.simplify(sp.factor(Gamma1)))
print(Gamma1)

(chi_max*eta_E*c(x, ttilde) + 1)*(Omega*c_ref*nu*x*c(x, ttilde) + Omega*c_ref*x*c(x, ttilde) + 3*nu*x*Derivative(u(x, ttilde), x) - 3*nu*u(x, ttilde) - 3*x*Derivative(u(x, ttilde), x))/(3*Omega*c_ref*x*(nu + 1)*(2*nu - 1))


In [ ]:
# gamma1 in comsol, for converting gamma1 to comsol compatible I had to change it a bit
(chi_max*eta_E*c + 1)*(Omega*c_ref*nu*x*c + Omega*c_ref*x*c + 3*nu*x*ux - 3*nu*u - 3*x*ux)/(3*Omega*c_ref*x*(nu + 1)*(2*nu - 1))

In [37]:
f1 = sp.factor(sp.simplify(sp.factor(f1)))
print(f1)

-(x*Derivative(u(x, ttilde), x) - u(x, ttilde))*(chi_max*eta_E*c(x, ttilde) + 1)/(Omega*c_ref*x**2*(nu + 1))


In [ ]:
# f1 in comsol
-(x*ux - u)*(chi_max*eta_E*c + 1)/(Omega*c_ref*(x^2)*(nu + 1))

# Second PDE

In [44]:
jt = -sp.diff(ct, rt, 1) + (om**2)*E_0*c_ref*ct*sp.diff(sigmat_hyd, rt, 1)/(R_g*T) # non dimensional mass flux
pde2_lhs = sp.Eq( sp.diff(ct, tt, 1) + sp.diff(jt*rt, rt, 1)/rt, 0).lhs
sp.factor(sp.simplify(sp.factor(pde2_lhs)))

-(-2*E_0*Omega**2*c_ref*chi_max*eta_E*x**3*c(x, ttilde)**2*Derivative(c(x, ttilde), (x, 2)) - 4*E_0*Omega**2*c_ref*chi_max*eta_E*x**3*c(x, ttilde)*Derivative(c(x, ttilde), x)**2 - 2*E_0*Omega**2*c_ref*chi_max*eta_E*x**2*c(x, ttilde)**2*Derivative(c(x, ttilde), x) - E_0*Omega**2*c_ref*x**3*c(x, ttilde)*Derivative(c(x, ttilde), (x, 2)) - E_0*Omega**2*c_ref*x**3*Derivative(c(x, ttilde), x)**2 - E_0*Omega**2*c_ref*x**2*c(x, ttilde)*Derivative(c(x, ttilde), x) + E_0*Omega*chi_max*eta_E*x**3*c(x, ttilde)**2*Derivative(u(x, ttilde), (x, 3)) + 3*E_0*Omega*chi_max*eta_E*x**3*c(x, ttilde)*Derivative(c(x, ttilde), x)*Derivative(u(x, ttilde), (x, 2)) + E_0*Omega*chi_max*eta_E*x**3*c(x, ttilde)*Derivative(c(x, ttilde), (x, 2))*Derivative(u(x, ttilde), x) + E_0*Omega*chi_max*eta_E*x**3*Derivative(c(x, ttilde), x)**2*Derivative(u(x, ttilde), x) + 2*E_0*Omega*chi_max*eta_E*x**2*c(x, ttilde)**2*Derivative(u(x, ttilde), (x, 2)) + E_0*Omega*chi_max*eta_E*x**2*c(x, ttilde)*u(x, ttilde)*Derivative(c(x, tti

In [47]:
Gamma2 = jt
Gamma2 = sp.factor(sp.simplify(sp.factor(jt)))
Gamma2

-(-2*E_0*Omega**2*c_ref*chi_max*eta_E*x**2*c(x, ttilde)**2*Derivative(c(x, ttilde), x) - E_0*Omega**2*c_ref*x**2*c(x, ttilde)*Derivative(c(x, ttilde), x) + E_0*Omega*chi_max*eta_E*x**2*c(x, ttilde)**2*Derivative(u(x, ttilde), (x, 2)) + E_0*Omega*chi_max*eta_E*x**2*c(x, ttilde)*Derivative(c(x, ttilde), x)*Derivative(u(x, ttilde), x) + E_0*Omega*chi_max*eta_E*x*c(x, ttilde)**2*Derivative(u(x, ttilde), x) + E_0*Omega*chi_max*eta_E*x*c(x, ttilde)*u(x, ttilde)*Derivative(c(x, ttilde), x) - E_0*Omega*chi_max*eta_E*c(x, ttilde)**2*u(x, ttilde) + E_0*Omega*x**2*c(x, ttilde)*Derivative(u(x, ttilde), (x, 2)) + E_0*Omega*x*c(x, ttilde)*Derivative(u(x, ttilde), x) - E_0*Omega*c(x, ttilde)*u(x, ttilde) + 6*R_g*T*nu*x**2*Derivative(c(x, ttilde), x) - 3*R_g*T*x**2*Derivative(c(x, ttilde), x))/(3*R_g*T*x**2*(2*nu - 1))

In [46]:
print(Gamma2)

-(-2*E_0*Omega**2*c_ref*chi_max*eta_E*x**2*c(x, ttilde)**2*Derivative(c(x, ttilde), x) - E_0*Omega**2*c_ref*x**2*c(x, ttilde)*Derivative(c(x, ttilde), x) + E_0*Omega*chi_max*eta_E*x**2*c(x, ttilde)**2*Derivative(u(x, ttilde), (x, 2)) + E_0*Omega*chi_max*eta_E*x**2*c(x, ttilde)*Derivative(c(x, ttilde), x)*Derivative(u(x, ttilde), x) + E_0*Omega*chi_max*eta_E*x*c(x, ttilde)**2*Derivative(u(x, ttilde), x) + E_0*Omega*chi_max*eta_E*x*c(x, ttilde)*u(x, ttilde)*Derivative(c(x, ttilde), x) - E_0*Omega*chi_max*eta_E*c(x, ttilde)**2*u(x, ttilde) + E_0*Omega*x**2*c(x, ttilde)*Derivative(u(x, ttilde), (x, 2)) + E_0*Omega*x*c(x, ttilde)*Derivative(u(x, ttilde), x) - E_0*Omega*c(x, ttilde)*u(x, ttilde) + 6*R_g*T*nu*x**2*Derivative(c(x, ttilde), x) - 3*R_g*T*x**2*Derivative(c(x, ttilde), x))/(3*R_g*T*x**2*(2*nu - 1))


In [ ]:
# gamma2 in comsol
-(-2*E_0*(Omega^2)*c_ref*chi_max*eta_E*(x^2)*(c^2)*cx - E_0*(Omega^2)*c_ref*(x^2)*c*cx + E_0*Omega*chi_max*eta_E*(x^2)*(c^2)*uxx + E_0*Omega*chi_max*eta_E*(x^2)*c*cx*ux + E_0*Omega*chi_max*eta_E*x*(c^2)*ux + E_0*Omega*chi_max*eta_E*x*c*u*cx - E_0*Omega*chi_max*eta_E*(c^2)*u + E_0*Omega*x^2*c*uxx + E_0*Omega*x*c*ux- E_0*Omega*c*u + 6*R_g*T*nu*(x^2)*cx - 3*R_g*T*(x^2)*cx)/(3*R_g*T*(x^2)*(2*nu - 1))

In [49]:
# while entering f2 in COMSOL i will ignore the time derivative term and treat it seperately
f2 = sp.diff(Gamma2, rt, 1) - pde2_lhs
f2 = sp.factor(sp.simplify(sp.factor(f2)))
f2

-(2*E_0*Omega**2*c_ref*chi_max*eta_E*x**2*c(x, ttilde)**2*Derivative(c(x, ttilde), x) + E_0*Omega**2*c_ref*x**2*c(x, ttilde)*Derivative(c(x, ttilde), x) - E_0*Omega*chi_max*eta_E*x**2*c(x, ttilde)**2*Derivative(u(x, ttilde), (x, 2)) - E_0*Omega*chi_max*eta_E*x**2*c(x, ttilde)*Derivative(c(x, ttilde), x)*Derivative(u(x, ttilde), x) - E_0*Omega*chi_max*eta_E*x*c(x, ttilde)**2*Derivative(u(x, ttilde), x) - E_0*Omega*chi_max*eta_E*x*c(x, ttilde)*u(x, ttilde)*Derivative(c(x, ttilde), x) + E_0*Omega*chi_max*eta_E*c(x, ttilde)**2*u(x, ttilde) - E_0*Omega*x**2*c(x, ttilde)*Derivative(u(x, ttilde), (x, 2)) - E_0*Omega*x*c(x, ttilde)*Derivative(u(x, ttilde), x) + E_0*Omega*c(x, ttilde)*u(x, ttilde) + 6*R_g*T*nu*x**3*Derivative(c(x, ttilde), ttilde) - 6*R_g*T*nu*x**2*Derivative(c(x, ttilde), x) - 3*R_g*T*x**3*Derivative(c(x, ttilde), ttilde) + 3*R_g*T*x**2*Derivative(c(x, ttilde), x))/(3*R_g*T*x**3*(2*nu - 1))

In [50]:
print(f2)

-(2*E_0*Omega**2*c_ref*chi_max*eta_E*x**2*c(x, ttilde)**2*Derivative(c(x, ttilde), x) + E_0*Omega**2*c_ref*x**2*c(x, ttilde)*Derivative(c(x, ttilde), x) - E_0*Omega*chi_max*eta_E*x**2*c(x, ttilde)**2*Derivative(u(x, ttilde), (x, 2)) - E_0*Omega*chi_max*eta_E*x**2*c(x, ttilde)*Derivative(c(x, ttilde), x)*Derivative(u(x, ttilde), x) - E_0*Omega*chi_max*eta_E*x*c(x, ttilde)**2*Derivative(u(x, ttilde), x) - E_0*Omega*chi_max*eta_E*x*c(x, ttilde)*u(x, ttilde)*Derivative(c(x, ttilde), x) + E_0*Omega*chi_max*eta_E*c(x, ttilde)**2*u(x, ttilde) - E_0*Omega*x**2*c(x, ttilde)*Derivative(u(x, ttilde), (x, 2)) - E_0*Omega*x*c(x, ttilde)*Derivative(u(x, ttilde), x) + E_0*Omega*c(x, ttilde)*u(x, ttilde) + 6*R_g*T*nu*x**3*Derivative(c(x, ttilde), ttilde) - 6*R_g*T*nu*x**2*Derivative(c(x, ttilde), x) - 3*R_g*T*x**3*Derivative(c(x, ttilde), ttilde) + 3*R_g*T*x**2*Derivative(c(x, ttilde), x))/(3*R_g*T*x**3*(2*nu - 1))


In [ ]:
# f2 in comsol
-(2*E_0*(Omega^2)*c_ref*chi_max*eta_E*(x^2)*(c^2)*cx + E_0*(Omega^2)*c_ref*(x^2)*c*cx - E_0*Omega*chi_max*eta_E*(x^2)*(c^2)*uxx - E_0*Omega*chi_max*eta_E*(x^2)*c*cx*ux - E_0*Omega*chi_max*eta_E*x*(c^2)*ux - E_0*Omega*chi_max*eta_E*x*c*u*cx + E_0*Omega*chi_max*eta_E*(c^2)*u - E_0*Omega*(x^2)*c*uxx - E_0*Omega*x*c*ux + E_0*Omega*c*u - 6*R_g*T*nu*(x^2)*cx + 3*R_g*T*(x^2)*cx)/(3*R_g*T*(x^3)*(2*nu - 1))